<a href="https://colab.research.google.com/github/nalrob/Me/blob/main/ARV_IsaacSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Qué es Cortex y cómo se integra con Isaac Sim?

NVIDIA Isaac Sim™ es una aplicación de referencia creada en NVIDIA Omniverse que permite a los desarrolladores desarrollar, simular y probar robots controlados por IA en entornos virtuales basados ​​​​en física.

Pipeline de Cortex

1. Percepción: Los flujos sensoriales se procesan para obtener información sobre lo que hay en el mundo y dónde están esos objetos.

2. Modelado del mundo: esta información se escribe en la base de datos de USD. USD representa nuestra creencia mundial y captura toda la información disponible.

3. Monitoreo del estado lógico: monitorea mundo y registran el estado lógico actual del entorno. El estado lógico incluye información discreta, como si una puerta está abierta o cerrada.

4. Toma de decisiones: en función del modelo del mundo y del estado lógico, el sistema debe decidir qué hacer. Lo que se debe hacer se define mediante los comandos disponibles a través de la API de comandos expuesta.

5. API de comandos (políticas): el comportamiento está determinado por políticas, y cada política está regida por un conjunto de parámetros.

6. Control: Y finalmente, el control de bajo nivel sincroniza el estado robótico interno con el robot físico para su ejecución en tiempo real.


Integración

  >El *Mundo de comandos*- (2)mundo, (3)lógica, (4)decisiones, (4)comandos ocurre desde la mente del robot y se simula en la API.
  
  >La *(1)Percepcion y (6)Control desde ROS*

# I. Instancia y Entorno

### I.I. Creación de una instancia de Isaac Sim.

In [ ]:
from isaacsim import SimulationApp

# Crear una instancia de la simulación (puede ser en modo headless o con GUI)
simulation_app = SimulationApp({"headless": False})  # False para visualizar, True para ejecutar sin GUI

### I.II. Creación de un entorno

Cómo crear un mundo y agregar un robot se configura en `isaacsim/cortex/framework/cortex_world.py
`

In [ ]:
from isaacsim.cortex.framework.cortex_world import CortexWorld
from isaacsim.cortex.framework.robot import add_franka_to_stage

# Crear el mundo de simulación de Cortex
world = CortexWorld()

# Agregar un robot Franka a la escena
robot = world.add_robot(add_ur20_to_stage(name="ur20", prim_path="/World/ur20"))

# Agregar un plano de suelo a la simulación
world.scene.add_default_ground_plane()

Como crear un comportamiento, y ejecutarlo.

In [ ]:
def step(self, render: bool = True, step_sim: bool = True) -> None:
    if self._task_scene_built:
        ...
        if self.is_playing():
            # **Flujo de Cortex:**
            # 1. Procesar los monitores de estado lógico antes del paso de simulación.
            # 2. Tomar decisiones basadas en el estado lógico (envía comandos a los comandantes del robot).
            # 3. Ejecutar los comandantes del robot para procesar esos comandos.

            # Paso 1: Procesar los monitores de estado lógico antes de la simulación.
            for ls_monitor in self._logical_state_monitors.values():
                ls_monitor.pre_step()

            # Paso 2: Evaluar los comportamientos antes de la simulación.
            for behavior in self._behaviors.values():
                behavior.pre_step()

            # Paso 3: Ejecutar los pasos previos de los robots antes de la simulación.
            for robot in self._robots.values():
                robot.pre_step()
        ...


Despues de configurar escena se debe iniciar loop de simulación.

In [ ]:
# Iniciar la simulación con el mundo de Cortex
world.run(simulation_app)

In [ ]:
# Cerrar la simulación cuando termine
simulation_app.close()

# II. Conceptos clave

### II.I. Command API

### II.II. Cálculos de la matriz de rotación

### II.II Red de toma de decisiones

# III. Implementación de un Robot con Cortex

Rutinas clave se implementan en ...

III.I. Stand By: AGV espera en estación de carga energética esperando orden​

III.II. Identificación: AGV recibe orden, navega a la posición e identifica la tarima a recoger​



III.III. Traslado: El AGV levanta la tarima, navega a la posición de deposito​


III.IV. Deposito: El AGV identifica si ya hay tarima en esa posición, de ser así busca otro espacio para dejar la tarima en espera​.

III.V. Reasignación: El AGV navega hacia la estación de carga energética, pero si está cerca de una tarima a recoger, se le asigna y repite las rutinas anteriores.​


III.VI. Stand By: Si AGV no estaba en la estación y llega a la estación, se conecta y entra en modo de espera de orden.

# IV. Integración Software & Hardware

### Integración de sensores

Componentes clave:
>Base

>Gripper

>Brazo

Hardware:
>UR20

>R14 (elevador)

>Robotics Vision

>E10 Slam

>TM12 (brazo)

### Estado Lógico

El marco de decisión completo se implementa en:
`isaacsim.cortex.framework/isaacsim/cortex/framework/df.py`

In [ ]:
# Importar la aplicación de simulación de Isaac Sim
from isaacsim import SimulationApp
simulation_app = SimulationApp({"headless": False})  # Ejecutar con renderizado activo

# Importar módulos de Cortex para la simulación y control del robot
from isaacsim.cortex.framework.cortex_world import CortexWorld
from isaacsim.cortex.framework.df import DfNetwork, DfDecider
from isaacsim.cortex.framework.dfb import DfContext
from isaacsim.cortex.framework.robot import add_franka_to_stage

# Definir un contexto personalizado para manejar el estado lógico del robot
class CustomContext(DfContext):
    def __init__(self, robot):
        super().__init__(robot)

    def reset(self):
        # Se ejecuta antes de que inicie el comportamiento, útil para inicializar estados
        self.has_work = False

    def monitor_work(self):
        # Método para monitorear si hay trabajo pendiente (debe ser implementado)
        ...

# Definir un Decider para gestionar las acciones del robot
class Dispatch(DfDecider):
    def __init__(self):
        super().__init__()
        self.add_child("go_home", GoHome())  # Acción para regresar a casa
        self.add_child("do_work", DoWork())  # Acción para realizar trabajo

    def decide(self):
        # Lógica de decisión basada en el estado del contexto
        if self.context.has_work:
            return DfDecision("do_work")  # Si hay trabajo, ejecutar la acción correspondiente
        else:
            return DfDecision("go_home")  # De lo contrario, regresar a casa

# Crear el mundo de simulación Cortex
world = CortexWorld()

# Agregar el robot Franka al entorno de simulación
robot = world.add_robot(add_franka_to_stage(name="franka", prim_path="/World/franka"))

# Agregar un plano de suelo predeterminado a la escena
world.scene.add_default_ground_plane()

# Crear la red de decisión con el decider y el contexto personalizado
decider_network = DfNetwork(Dispatch(), context=CustomContext())

# Agregar la red de decisión al mundo
world.add_decider_network(decider_network)

# Iniciar la simulación
world.run(simulation_app)

# Cerrar la simulación al finalizar
simulation_app.close()